In [1]:
using StanModels
using CmdStan, StanMCMCChain

ProjDir = rel_path_s("..", "scripts", "12")
cd(ProjDir)

d = CSV.read(rel_path( "..", "data",  "Kline.csv"), delim=';');
size(d) # Should be 10x5

(10, 5)

New col log_pop, set log() for population data

In [2]:
d[:log_pop] = map((x) -> log(x), d[:population]);
d[:society] = 1:10;
first(d[[:culture, :population, :log_pop, :society]], 5)

m12_6 = "
data {
    int N;
    int N_societies;
    int total_tools[N];
    real logpop[N];
    int society[N];
}
parameters{
    real a;
    real bp;
    vector[N_societies] a_society;
    real<lower=0> sigma_society;
}
model{
    vector[N_societies] mu;
    sigma_society ~ cauchy( 0 , 1 );
    a_society ~ normal( 0 , sigma_society );
    bp ~ normal( 0 , 1 );
    a ~ normal( 0 , 10 );
    for ( i in 1:N ) {
        mu[i] = a + a_society[society[i]] + bp * logpop[i];
        mu[i] = exp(mu[i]);
    }
    total_tools ~ poisson( mu );
}
";

Define the Stanmodel and set the output format to :mcmcchain.

In [3]:
stanmodel = Stanmodel(name="m12.6",  model=m12_6, output_format=:mcmcchain);


File /Users/rob/.julia/dev/StanModels/scripts/12/tmp/m12.6.stan will be updated.



Input data for cmdstan

In [4]:
m12_6_data = Dict("N" => size(d, 1),"N_societies" => 10,
"total_tools" => d[:total_tools], "logpop" => d[:log_pop],
"society" => d[:society]);

Sample using cmdstan

In [5]:
rc, chn, cnames = stan(stanmodel, m12_6_data, ProjDir, diagnostics=false, summary=false, CmdStanDir=CMDSTAN_HOME);

Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
Exception: normal_lpdf: Scale parameter is 0, but must be > 0!  (in '/Users/rob/.julia/dev/StanModels/scripts/12/tmp/m12.6.stan' at line 17)

If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.



Describe the draws

In [6]:
describe(chn)

Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
                   Mean            SD         Naive SE        MCSE         ESS    
         lp__  928.006257500  3.0618467579 0.048412048006 0.0980132660  975.881714
accept_stat__    0.930488503  0.0956354165 0.001512128706 0.0019565594 1000.000000
   stepsize__    0.056599500  0.0031152208 0.000049255966 0.0004987722   39.009752
  treedepth__    5.796000000  0.4684430165 0.007406734431 0.0121939455 1000.000000
 n_leapfrog__   66.154000000 27.3006706081 0.431661503859 0.6449926257 1000.000000
  divergent__    0.000000000  0.0000000000 0.000000000000 0.0000000000         NaN
     energy__ -921.543517000  3.9773669610 0.062887693436 0.1156483820 1000.000000
            a    1.086289644  0.7284432427 0.011517698966 0.0239918454  921.858214
           bp    0.261935929  0.0785957315 0.001242707630 0.0025971780  915.787048
  a_society.1   -0.195565764  0.2429783309 0.003841824

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*